---
title: "Access and Analyse EOPF STAC Zarr Data with R"
format:
  md:
    df-print: tibble
execute:
  cache: true
editor_options: 
  chunk_output_type: console
jupyter:
    kernelspec:
        name: "ir43"
        language: "R"
        display_name: "R 4.3.2"
---

# Table of Contents

- [Introduction](#introduction)
- [Prerequisites](#prerequisites)
  - [Dependencies](#dependencies)
  - [Fixes to the Rarr package](#fixes-to-the-rarr-package)
- [Access Zarr data from the STAC catalog](#access-zarr-data-from-the-stac-catalog)
- [Read Zarr data](#read-zarr-data)
  - [Coordinates](#coordinates)
  - [Different resolutions](#different-resolutions)
- [Examples](#examples)
  - [Sentinel-1](#sentinel-1)
  - [Sentinel-2](#sentinel-2)
  - [Sentinel-3](#sentinel-3)
  
# Introduction

This tutorial will explore how to access and analyse Zarr data from the [EOPF Sample Service STAC catalog](https://stac.browser.user.eopf.eodc.eu/) programmatically using R.

# Prerequisites

An R environment is required to follow this tutorial, with R version >= 4.1.0. We recommend using either [RStudio](https://posit.co/download/rstudio-desktop/) or [Positron](https://posit.co/products/ide/positron/) (or a cloud computing environment) and making use of [RStudio projects](https://support.posit.co/hc/en-us/articles/200526207-Using-RStudio-Projects) for a self-contained coding environment.

## Dependencies

We will use the following packages in this tutorial: [`rstac`](https://brazil-data-cube.github.io/rstac/) (for accessing the STAC catalog), [`tidyverse`](https://tidyverse.tidyverse.org/) (for data manipulation), [`stars`](https://r-spatial.github.io/stars/)) (for working with spatiotemporal data) , and [`terra`](https://rspatial.github.io/terra/index.html) (for manipulating spatial data in raster format). You can install them directly from CRAN:

In [ ]:
#| eval: false
install.packages("rstac")
install.packages("tidyverse")
install.packages("stars")
install.packages("terra")

We will also use the `Rarr` package to read Zarr data. It must be installed from Bioconductor, so first install the `BiocManager` package:

In [ ]:
#| eval: false
install.packages("BiocManager")

Then, use this package to install `Rarr`:

In [ ]:
#| eval: false
BiocManager::install("Rarr")

Finally, load the packages into your environment:

In [ ]:
library(rstac)
library(tidyverse)
library(Rarr)
library(stars)
library(terra)

## Fixes to the `Rarr` package

We will use functions from the `Rarr` package to read and analyse Zarr data. Unfortunately, there is currently a bug in this package, causing it to parse the EOPF Sample Service data URLs incorrectly -- it has been fixed and will be updated in the next release of `Rarr`. In the meantime, we will write our own version of this URL parsing function and use it instead of the one in `Rarr`.

In [ ]:
.url_parse_other <- function(url) {
  parsed_url <- httr::parse_url(url)
  bucket <- gsub(
    x = parsed_url$path, pattern = "^/?([[a-z0-9\\:\\.-]*)/.*",
    replacement = "\\1", ignore.case = TRUE
  )
  object <- gsub(
    x = parsed_url$path, pattern = "^/?([a-z0-9\\:\\.-]*)/(.*)",
    replacement = "\\2", ignore.case = TRUE
  )
  hostname <- paste0(parsed_url$scheme, "://", parsed_url$hostname)

  if (!is.null(parsed_url$port)) {
    hostname <- paste0(hostname, ":", parsed_url$port)
  }

  res <- list(
    bucket = bucket,
    object = object,
    region = "auto",
    hostname = hostname
  )
  return(res)
}

assignInNamespace(".url_parse_other", .url_parse_other, ns = "Rarr")

This function overwrites the existing one in `Rarr`, and allows us to continue with the analysis.

If you try to run some of the examples below and receive a timeout error, please ensure that you have run the above code block.

# Access Zarr data from the STAC Catalog

The first step of accessing Zarr data is to understand the assets within the EOPF Sample Service STAC catalog. The [first tutorial](/eopf_stac_access.qmd) goes into detail on this, so we recommend reviewing it if you have not already.

For the first part of this tutorial, we will be using data from the [Sentinel-2 Level-2A Collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a). We fetch the "product" asset under a given item, and can look at its URL:

In [ ]:
s2_l2a_item <- stac("https://stac.core.eopf.eodc.eu/") |>
  collections(collection_id = "sentinel-2-l2a") |>
  items(feature_id = "S2B_MSIL2A_20250530T101559_N0511_R065_T32TPT_20250530T130924") |>
  get_request()

s2_l2a_product <- s2_l2a_item |>
  assets_select(asset_names = "product")

s2_l2a_product_url <- s2_l2a_product |>
  assets_url()

s2_l2a_product_url

The product is the "top level" Zarr asset, which contains the full Zarr product hierarchy. We can use `zarr_overview()` to get an overview of it, setting `as_data_frame` to `TRUE` so that we can see the entries in a data frame instead of printed directly to the console. Each entry is a Zarr array; we remove `product_url` from the path to get a better idea of what each array is. Since this is something we will want to do multiple times throughout the tutorial, we create a helper function for this.

In [ ]:
derive_store_array <- function(store, product_url) {
  store |>
    mutate(array = str_remove(path, product_url)) |>
    relocate(array, .before = path)
}

zarr_store <- s2_l2a_product_url |>
  zarr_overview(as_data_frame = TRUE) |>
  derive_store_array(s2_l2a_product_url)

zarr_store

This shows us the path to access the Zarr array, the number of chunks it contains, the type of data, as well as its dimensions and chunking structure.

We can also look at overviews of individual arrays. First, let's narrow down to measurements taken at 20-metre resolution:

In [ ]:
r20m <- zarr_store |>
  filter(str_starts(array, "/measurements/reflectance/r20m/"))

r20m

Then, we select the B02 array and examine its dimensions and chuning:

In [ ]:
r20m |>
  filter(str_ends(array, "b02")) |>
  select(path, nchunks, dim, chunk_dim) |>
  as.list()

We can also see an overview of individual arrays using `zarr_overview()`. With the default setting (where `as_data_frame` is `FALSE`), this prints information on the array directly to the console, in a more digestible way:

In [ ]:
r20m_b02 <- r20m |>
  filter(str_ends(array, "b02")) |>
  pull(path)

r20m_b02 |>
  zarr_overview()

The above overview tells us that the data is two-dimensional, with dimensions 5490 x 5490. Zarr data is split up into **chunks**, which are smaller, independent piece of the larger array. Chunks can be accessed individually, without loading the entire array. In this case, there are 36 chunks in total, with 6 along each of the dimensions, each of size 915 x 915.

# Read Zarr data

To read in Zarr data, we use `read_zarr_array()`, and can pass a list to the `index` argument, describing which elements we want to extract along each dimension. Since this array is two-dimensional, we can think of the dimensions as rows and columns of the data. For example, to select the first 10 rows and the first 5 columns:

In [ ]:
r20m_b02 |>
  read_zarr_array(index = list(1:10, 1:5))

## Coordinates

Similarly, we can read in the `x` and `y` coordinates corresponding to data at 10m resolution. These `x` and `y` coordinates do not correspond to latitude and longitude---to understand the coordinate reference system used in each data set, we access the `proj:code` property of the STAC item. In this case, the coordinate reference system is [EPSG:32626](https://epsg.io/32626), which represents metres from the UTM zone's origin.

In [ ]:
s2_l2a_item[["properties"]][["proj:code"]]

We can see that `x` and `y` are one dimensional:

In [ ]:
r20m_x <- r20m |>
  filter(str_ends(array, "x")) |>
  pull(path)

r20m_x |>
  zarr_overview()

r20m_y <- r20m |>
  filter(str_ends(array, "y")) |>
  pull(path)

r20m_y |>
  zarr_overview()

Which means that, when combined, they form a grid that describes the location of each point in the 2-dimensional measurements, such as B02. We will go into this more in the examples below.

The `x` and `y` dimensions can be read in using the same logic: by describing which elements we want to extract. Since there is only one dimension, we only need to supply one entry in the indexing list:

In [ ]:
r20m_x |>
  read_zarr_array(list(1:5))

Or, we can read in the whole array (by not providing any elements to `index`) and view its first few values with `head()`. Of course, reading in the whole array, rather than a small section of it, will take longer.

In [ ]:
r20m_x |>
  read_zarr_array() |>
  head(5)

r20m_y |>
  read_zarr_array() |>
  head(5)

## Different resolutions

With EOPF data, some measurements are available at multiple resolutions. For example, we can see that the B02 spectral band is available at 10m, 20m, and 60m resolution:

In [ ]:
b02 <- zarr_store |>
  filter(str_starts(array, "/measurements/reflectance"), str_ends(array, "b02"))

b02 |>
  select(array)

The resolution affects the dimensions of the data: when measurements are taken at a higher resolution, there will be more data. We can see here that there is more data for the 10m resolution than the 20m resolution (recall, its dimensions are 5490 x 5490), and less for the 60m resolution:

In [ ]:
b02 |>
  filter(array == "/measurements/reflectance/r10m/b02") |>
  pull(dim)

b02 |>
  filter(array == "/measurements/reflectance/r60m/b02") |>
  pull(dim)

# Examples

The following sections show examples from each of the Sentinel missions.

## Sentinel-1

The first example looks at [Sentinel-1 Level 2 Ocean (OCN) data](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-1-l2-ocn), which consists of data for oceanographic study, such as monitoring sea surface conditions, detecting oil spills, and studying ocean currents. This example will show how to access and plot Wind Direction data.

First, select the relevant collection and item from STAC:

In [ ]:
l2_ocn <- stac("https://stac.core.eopf.eodc.eu/") |>
  collections(collection_id = "sentinel-1-l2-ocn") |>
  items(feature_id = "S1A_IW_OCN__2SDV_20250604T193923_20250604T193948_059501_0762FA_C971") |>
  get_request()

l2_ocn

We can look at each of the assets' titles to understand what the item contains:

In [ ]:
l2_ocn |>
  pluck("assets") |>
  map("title")

We are interested in the "Ocean Wind field" data, and will hold onto the `owi` key for now.

To access all of the `owi` data, we get the "product" asset and then the full Zarr store, again using our helper function to extract array information from the full array path:

In [ ]:
l2_ocn_url <- l2_ocn |>
  assets_select(asset_names = "product") |>
  assets_url()

l2_ocn_store <- l2_ocn_url |>
  zarr_overview(as_data_frame = TRUE) |>
  derive_store_array(l2_ocn_url)

l2_ocn_store

Next, we filter to access `owi` measurement data only:

In [ ]:
l2_ocn_store |>
  filter(str_starts(array, "/owi"), str_detect(array, "measurements"))

Since all of these arrays start with `/owi/S01SIWOCN_20250604T193923_0025_A340_C971_0762FA_VV/measurements/`, we can remove that to get a clearer idea of what each array is:

In [ ]:
owi <- l2_ocn_store |>
  filter(str_starts(array, "/owi"), str_detect(array, "measurements")) |>
  mutate(array = str_remove(array, "/owi/S01SIWOCN_20250604T193923_0025_A340_C971_0762FA_VV/measurements/"))

owi

We are interested in `wind_direction`, as well as the coordinate arrays (`latitude` and `longitude`). We can get an overview of the arrays' dimensions and structures:

In [ ]:
owi |>
  filter(array == "wind_direction") |>
  pull(path) |>
  zarr_overview()

owi |>
  filter(array == "latitude") |>
  pull(path) |>
  zarr_overview()

owi |>
  filter(array == "longitude") |>
  pull(path) |>
  zarr_overview()

Here, we can see that all of the arrays are of the same shape: 167 x 255, with only one chunk. Since these are small, we can read all of the data in at once.

In [ ]:
owi_wind_direction <- owi |>
  filter(array == "wind_direction") |>
  pull(path) |>
  read_zarr_array()

owi_wind_direction[1:5, 1:5]

owi_lat <- owi |>
  filter(array == "latitude") |>
  pull(path) |>
  read_zarr_array()

owi_lat[1:5, 1:5]

owi_long <- owi |>
  filter(array == "longitude") |>
  pull(path) |>
  read_zarr_array()

owi_lat[1:5, 1:5]

Note that both `longitude` and `latitude` are 2-dimensional arrays, and they are not evenly spaced. Rather, the data grid is **curvilinear** --- it has grid lines that are not straight, and there is a longitude and latitude for every pixel of the other layers (i.e., `wind_direction`). This format is very common in satellite data.

We use functions from the `stars` package, loaded earlier, to format the data for visualisation. `stars` is specifically designed for reading, manipulating, and plotting spatiotemporal data, such as satellite data.

The function `st_as_stars()` is used to get our data into the correct format for visualisation:

In [ ]:
owi_stars <- st_as_stars(wind_direction = owi_wind_direction) |>
  st_as_stars(curvilinear = list(X1 = owi_long, X2 = owi_lat))

Getting the data into this format is also beneficial because it allows for a quick summary of the data and its attributes, providing information such as the median and mean `wind_direction`, the number of `NA`s, and information on the grid:

In [ ]:
owi_stars

Finally, we can plot this object:

In [ ]:
plot(owi_stars, main = "Wind Direction", as_points = FALSE, axes = TRUE, breaks = "equal", col = hcl.colors)

## Sentinel-2

EOPF Zarr Assets include quicklook RGB composites, which are readily viewable representations of the satellite image. We will open the 10-metre resolution quicklook and visualise it. This is available as an asset, so we can access it directly from the STAC item.

In [ ]:
tci_10m_asset <- s2_l2a_item |>
  assets_select(asset_names = "TCI_10m")

tci_10m_url <- tci_10m_asset |>
  assets_url()

tci_10m_url |>
  zarr_overview()

From the overview, we can see that the quicklook array has three dimensions to it, each of size 10980 x 10980. The three dimensions correspond to red, green, and blue spectral bands (B04, B03, and B02, respectively), since this is an RGB composite. This information is also available by looking at the assets' bands:

In [ ]:
s2_l2a_item[["assets"]][["TCI_10m"]][["bands"]] |>
  map_dfr(as_tibble)

We can read in a small chunk of the array to get an idea of its shape, using the same indexing process we've used before. Note that we want to select _all_ of the bands (the first dimension listed). Rather than writing `1:3`, we can simply use `NULL` as the first dimension, indicating to get all data at this dimension. To preview the data, we will just get the first 2 entries (in each dimension) along the three bands.

In [ ]:
tci_10m_preview <- tci_10m_url |>
  read_zarr_array(list(NULL, 1:2, 1:2))

tci_10m_preview

For visualisation purposes, we need the data in a different configuration --- note the dimensions of the data:

In [ ]:
dim(tci_10m_preview)

Instead, we need to get it into e.g. 2 x 2 x 3, with the _third_ dimension reflecting the number of bands (or layers) To do this, we use the `aperm()` function to transpose an array, with argument `c(2, 3, 1)` -- moving the second dimension to the first, the third to the second, and the first to the third. Then, we can see that the dimensions of the array are correct:

In [ ]:
tci_10m_preview_perm <- tci_10m_preview |>
  aperm(c(2, 3, 1))

tci_10m_preview_perm

dim(tci_10m_preview_perm)

Let's read in the full TCI array to visualise it.

In [ ]:
tci_10m <- tci_10m_url |>
  read_zarr_array()

tci_10m_perm <- tci_10m |>
  aperm(c(2, 3, 1))

dim(tci_10m)

For visualisation, we use `terra`'s `plotRGB()` function, first converting the array into a raster object with `rast()`:

In [ ]:
tci_10m_perm |>
  rast() |>
  plotRGB()

We can do the same with the quicklook at the 60-metre resolution, showing the full visualisation process in a single step:

In [ ]:
zarr_store |>
  filter(array == "/quality/l2a_quicklook/r60m/tci") |>
  pull(path) |>
  read_zarr_array() |>
  aperm(c(2, 3, 1)) |>
  rast() |>
  plotRGB()

## Sentinel-3

Finally, we look at an example from the Sentinel-3 mission. The Sentinel-3 mission measures sea-surface topography and land- and sea-surface temperature and colour, in support of environmental and climate monitoring. The [Sentinel-3 OLCI L2 LFR](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-3-olci-l2-lfr?.language=en) product provides this data, computed for full resolution.

Again, we will access a specific item from this collection:

In [ ]:
l2_lfr <- stac("https://stac.core.eopf.eodc.eu/") |>
  collections(collection_id = "sentinel-3-olci-l2-lfr") |>
  items(feature_id = "S3A_OL_2_LFR____20250605T102430_20250605T102730_20250605T122455_0179_126_336_2160_PS1_O_NR_003") |>
  get_request()

l2_lfr

To access all of the data, we get the "product" asset and then the full Zarr store, again using our helper function to extract array information from the full array path:

In [ ]:
l2_lfr_url <- l2_lfr |>
  assets_select(asset_names = "product") |>
  assets_url()

l2_lfr_store <- l2_lfr_url |>
  zarr_overview(as_data_frame = TRUE) |>
  derive_store_array(l2_lfr_url)

l2_lfr_store

Next, we filter to access measurement data only:

In [ ]:
l2_lfr_measurements <- l2_lfr_store |>
  filter(str_starts(array, "/measurements")) |>
  mutate(array = str_remove(array, "/measurements/"))

l2_lfr_measurements

Of these, we are interested in `gifapar` as well as `longitude` and `latitude`. We can get an overview of the arrays’ dimensions and structures:

In [ ]:
l2_lfr_measurements |>
  filter(array == "gifapar") |>
  pull(path) |>
  zarr_overview()

l2_lfr_measurements |>
  filter(array == "longitude") |>
  pull(path) |>
  zarr_overview()

l2_lfr_measurements |>
  filter(array == "latitude") |>
  pull(path) |>
  zarr_overview()

Similar to the previous example, we can see that all of the arrays are of the same shape: 4091 x 4865. We read in all of the arrays:

In [ ]:
gifapar <- l2_lfr_measurements |>
  filter(array == "gifapar") |>
  pull(path) |>
  read_zarr_array()

gifapar_long <- l2_lfr_measurements |>
  filter(array == "longitude") |>
  pull(path) |>
  read_zarr_array()

gifapar_long[1:5, 1:5]

gifapar_lat <- l2_lfr_measurements |>
  filter(array == "latitude") |>
  pull(path) |>
  read_zarr_array()

gifapar_lat[1:5, 1:5]

Again, both `longitude` and `latitude` are unevenly spaced 2-dimensional arrays. This tells us that the data grid is curvilinear, and we use `st_as_stars()` to get our data into the correct format for visualisation:

In [ ]:
gifapar_stars <- st_as_stars(gifapar = gifapar) |>
  st_as_stars(curvilinear = list(X1 = gifapar_long, X2 = gifapar_lat))

gifapar_stars

Finally, we plot the GIFAPAR:

In [ ]:
plot(gifapar_stars, as_points = FALSE, axes = TRUE, breaks = "equal", col = hcl.colors)